# Brain Tumors MRIs Segmentation Masks vs CNN Grad-cam ++ Saliency Masks : 

# *Which Texture Features For Each Tumor Image Will Potential Radiomic Biomarkers Show Up?*

Used Datasets : • Masoud Nickparvar, Kaggle Brain Tumor Dataset, 2020.
                • SciDB Brain Tumor Dataset, SciDB, 2021.
Indrakumar K, and Ravikumar M. (2025). Brain Tumor Dataset: Segmentation & Classification [Data set]. Kaggle. https://doi.org/10.34740/KAGGLE/DSV/11957028

### Firstly, install dependencies...

In [ ]:
!pip install pyradiomics
!pip install SimpleITK

### And load the imports...

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import SimpleITK as sitk
from radiomics import featureextractor
from sklearn.feature_selection import VarianceThreshold, f_classif
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import shap
from keras.models import Sequential 
from keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from keras.metrics import Accuracy, Precision, F1Score
from keras.models import Model 
import random
import os
import seaborn as sns
from scipy import stats
from scipy.stats import wilcoxon
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.optimizers import Adam
from keras.applications.mobilenet_v2 import MobileNetV2

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

This segmentation dataset contains three types of brain tumors : glioma, meningioma, and pituitary. Each mask per image has already beem provided, where Pyradiomics's feature extractor will only focus on that masked region of the tumor (not focused on the background of the image), and find interesting biological details about its texture surrounding the region.

**This imaging dataset has already been preprocessed to have noise reduction and is normalized.**

In [ ]:
csv_file = "/kaggle/input/brain-tumor-classes/tumor_segment.csv"

data = pd.read_csv(csv_file)

class_distribution = data['class_name'].value_counts()
print(class_distribution)

plt.figure(figsize=(10,6))
class_distribution.plot(kind = 'bar', color='violet')
plt.title('Class Distribution')
plt.xlabel('Class Label')
plt.ylabel("Number of Images")
plt.xticks(rotation = 45)
plt.show()

In [ ]:
data.head()

#### **An example of a 2D Glioma tumor and its designated segmentation mask :**

In [ ]:
image_path_mri = "/kaggle/input/brain-tumor-dataset-segmentation-and-classification/DATASET/Segmentation/Glioma/enh_1841.png"
image_path_mask = "/kaggle/input/brain-tumor-dataset-segmentation-and-classification/DATASET/Segmentation/Glioma/enh_1841_mask.png"

img_mri = Image.open(image_path_mri)
img_mask = Image.open(image_path_mask)

plt.imshow(img_mri)    #MRI example
plt.axis('on')
plt.show


In [ ]:
plt.imshow(img_mask)
plt.axis('on')        #Mask example
plt.show

## Now lets extract those texture features from all of these masks from the dataset...

In [ ]:
seg_dir = "/kaggle/input/brain-tumor-dataset-segmentation-and-classification/DATASET/Segmentation"
tumors = ["Glioma", "Meningioma", "Pituitary tumor"] #What the folders say...

extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.enableAllFeatures() 
seg_images = []

for tumor in tumors:
    folder_path = os.path.join(seg_dir, tumor)

    for file in os.listdir(folder_path):
        if "mask" in file:  
            continue

        img_path = os.path.join(folder_path, file)
        mask_path = os.path.join(folder_path, file.replace('.png', '_mask.png'))
        image = sitk.ReadImage(img_path)
        mask = sitk.ReadImage(mask_path)
       
        feature_vector = extractor.execute(image, mask, label = 255) #Change size to 255 pixels
      
        feature_vector["tumor_type"] = tumor
        feature_vector["file_name"] = file
        seg_images.append(feature_vector) #Vectors

seg_images_df = pd.DataFrame(seg_images)

In [ ]:
seg_images_df.head()

In [ ]:
features = seg_images_df.drop(columns=['file_name', 'tumor_type'])
label = seg_images_df['tumor_type']

print(features.shape, label.shape) #The tumor is what we are associating everything else for

In [ ]:
cols_to_drop = [col for col in features.columns if col.startswith('diagnostics_')]
features = features.drop(columns=cols_to_drop)

features = features.select_dtypes(include=[np.number])
print(features.shape)

In [ ]:
selector = VarianceThreshold(threshold=0.01)
features_reduced = selector.fit_transform(features)

select = features.columns[selector.get_support()]
features = pd.DataFrame(features_reduced, columns = select)

print(features.shape)

In [ ]:
import pandas as pd
from xgboost import XGBClassifier

le = LabelEncoder()
label_encoded = le.fit_transform(label)

xgb = XGBClassifier(n_estimators=200, random_state=42, use_label_encoder=False, eval_metric='logloss') 
xgb.fit(features, label_encoded)

top_features = pd.DataFrame({
    'Feature': features.columns,
    'Importance': xgb.feature_importances_
}).sort_values('Importance', ascending=False)


print(top_features.head(15))

In [ ]:
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(features)

shap.summary_plot(shap_values, features)

In [ ]:
original_labels = le.inverse_transform(xgb.classes_) #fyi...
print(original_labels)

In [ ]:
plt.figure(figsize=(10, 6))
top_features.head(20).plot(kind='bar', x='Feature', y='Importance')
plt.title("Top 20 Radiomic Feature Importances")
plt.ylabel("Importance Score")
plt.show()

In [ ]:
F, p = f_classif(features, label)

anova_results = pd.DataFrame({
    "Feature": features.columns,    #anova also pulls out the same names as xgboost in a different order...
    "p-value": p,
    "F-score": F
}).sort_values("p-value").reset_index(drop=True)

anova_results.head(10)

In [ ]:
for shap_class, cls in zip(shap_values, xgb.classes_):
    print(f"Top features for class: {cls}")
    shap.summary_plot(shap_class, features, feature_names=features.columns)

## We can conclude that original_shape2D_MajorAxisLength, original_shape2D_MinorAxisLength, original_shape2D_PixelSurface, and original_shape2D_Elongation are the deciding factors that identifies a specific tumor based on its 2D textures.

But then what does a CNN classfier take from these ROIs (regions of interest that makes up a brain tumor identification), when it performs its own analysis looking for relevant texture features??

#### This classfication dataset has a new class 'notumor' where the classfier is expected to differentiate between healthy brains and ones with glioma, meningioma, or pituitary.

In [ ]:
csv_file = "/kaggle/input/brain-tumor-classes/tumor_class_train.csv"

data_class = pd.read_csv(csv_file)

class_distribution = data_class['class_name'].value_counts()
print(class_distribution)

plt.figure(figsize=(10,6))
class_distribution.plot(kind = 'bar', color='lime')
plt.title('Class Distribution')
plt.xlabel('Class Label')
plt.ylabel("Number of Images")
plt.xticks(rotation = 45)
plt.show()

In [ ]:
data_class.head()

### **The Four Classes of Brain Tumors :**

In [ ]:
selected_images = data_class.groupby('class_name')['image_path'].first().reset_index()

from PIL import Image

for _, row in selected_images.iterrows():
    className = row['class_name']
    imagePath = row['image_path']
    imageFile = os.path.join('/kaggle/input/brain-tumor-dataset-segmentation-and-classification/DATASET/classification/Training', imagePath)
    image = Image.open(imageFile)

    plt.figure()
    plt.imshow(image)
    plt.title(f'Class: {className}')
    plt.show()

## Now, lets preprocess these images so we can fine tune our pre-trained CNN without background artifacts...

In [ ]:
def add_gaussian_blur(img):
    img = cv2.GaussianBlur(img, (3,3), 0)
    return img

In [ ]:
train_df, validate_df = train_test_split(data_class, test_size=0.20, random_state=42)

train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

batch_size = 20
img_height = 224
img_width = 224

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10, 
    rescale=1./255, 
    shear_range=0.2, 
    zoom_range=0.10, 
    horizontal_flip=True, 
    brightness_range=[1.1, 1.2]
)

validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="/kaggle/input/brain-tumor-dataset-segmentation-and-classification/DATASET/classification/Training",
    x_col="image_path",  
    y_col="class_name",  
    target_size=(img_height, img_width), 
    batch_size=batch_size,
    shuffle=True ,
    class_mode="categorical" 
)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validate_df,
    directory="/kaggle/input/brain-tumor-dataset-segmentation-and-classification/DATASET/classification/Training",
    x_col="image_path",
    y_col="class_name",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False ,
    class_mode="categorical"
)

In [ ]:
test_dir = '/kaggle/input/brain-tumor-dataset-segmentation-and-classification/DATASET/classification/Testing'
test_csv_path = '/kaggle/input/brain-tumor-classes/tumor_class_test.csv'
test_df = pd.read_csv(test_csv_path)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    test_dir,
    x_col="image_path",
    y_col="class_name",
    target_size=(img_height, img_width),
    batch_size=1, #Better for plotting
    shuffle=False,
    class_mode="categorical"
)

Much better for the model.

In [ ]:
import matplotlib.pyplot as plt
images, labels = next(train_generator)

plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)   #New preprocessed MRI images
    plt.imshow(images[i])
    plt.title(f'Class: {labels[i]}')
    plt.axis('off')
plt.show()

In [ ]:
class_names = list(train_generator.class_indices.keys())
print("Class Names:", class_names)

## Now time to train our MobileNetV2 CNN model...

In [ ]:
input_shape = (img_height, img_width, 3)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
for layer in base_model.layers:
    layer.trainable = False
    
num_layers_to_unfreeze = 50

for layer in base_model.layers[-num_layers_to_unfreeze:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu')(x)  
predictions = Dense(4, activation='softmax')(x)  


model = tf.keras.Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(train_generator, epochs= 5, validation_data=validation_generator)

In [ ]:
first_test_predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)

In [ ]:
predicted_labels = np.argmax(first_test_predictions, axis=1)
true_labels = test_generator.classes
cm = confusion_matrix(true_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_generator.class_indices.keys())
disp.plot()
plt.show()

### Implementing Gradcam++

The difference bewteen Gradcam and Gradcam++ is that Gradcam++ focuses more on the pixels that make up the ROI and weighs them more, while Gradcam just puts weights on the whole ROI.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def grad_cam_plus_plus(model, img_array, layer_name):
    grad_model = tf.keras.models.Model(
        [model.inputs], 
        [model.get_layer(layer_name).output, model.output] #get conv layer feature maps
    )

    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_output) 

    numerator = grads ** 2
    denominator = 2 * grads ** 2 + tf.reduce_sum(conv_output * grads, axis=(1, 2), keepdims=True)   #weights on the pixels that make up the ROI  
    alpha = numerator / (denominator + 1e-10)

    weights = tf.reduce_sum(alpha * tf.nn.relu(grads), axis=(1, 2))
    cam = tf.reduce_sum(weights * conv_output, axis=-1).numpy()

    cam = np.maximum(cam, 0)
    cam = cam[0]  

    cam = cv2.resize(cam, (img_width, img_height))
    cam = cam / cam.max() #resize and normalize
    return cam

In [ ]:
last_conv_layer = None
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv_layer = layer.name
        break

print(last_conv_layer)

In [ ]:
test_generator.reset() #running predictions again...

predictions = model.predict(test_generator, steps=test_generator.samples // test_generator.batch_size + 1)
predicted_labels = np.argmax(predictions, axis=1)

true_labels = test_generator.classes 
class_names = list(test_generator.class_indices.keys())

In [ ]:
correct_images = []
correct_pred_labels = []

test_generator.reset()

for i in range(test_generator.samples):
    img_batch = next(test_generator)[0]
    img = img_batch[0]
    
    if predicted_labels[i] == true_labels[i]: 
        correct_images.append(img)
        correct_pred_labels.append(predicted_labels[i])

    if len(correct_images) >= 1200: 
        break

In [ ]:
correct_heatmaps = []

for img in correct_images:
    img_input = np.expand_dims(img, axis=0)
    heatmap = grad_cam_plus_plus(model, img_input, last_conv_layer)
    correct_heatmaps.append(heatmap)


In [ ]:
def get_saliency_mask(heatmap, tumor_mask=None, percentile=90):
    
    if tumor_mask is not None:  
        night_vision = np.percentile(heatmap[tumor_mask > 0], percentile)
        saliency_mask = (heatmap > night_vision) & (tumor_mask > 0)
    else:                                                     #like the segmented masks we dont pay attention towards the background
        night_vision = np.percentile(heatmap, percentile)
        saliency_mask = (heatmap > night_vision)

    return saliency_mask.astype(np.uint8)  

In [ ]:
import cv2
import numpy as np

def generate_pseudo_mask(img):
    gray = cv2.cvtColor((img * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY) #grayscale

    threshold_value, mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #print("threshold:", threshold_value)

    kernel = np.ones((5, 5), np.uint8) 
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    return (mask > 0).astype(np.uint8) 
    
saliency_masks = []

for i, (heatmap, pred_label) in enumerate(zip(correct_heatmaps, correct_pred_labels)):
    
    class_name = class_names[pred_label]
    
    if class_name != 'notumor': 
        tumor_mask = generate_pseudo_mask(correct_images[i]) 
    else:
        tumor_mask = None 

    mask = get_saliency_mask(heatmap, tumor_mask, percentile=90)
    saliency_masks.append(mask)

In [ ]:
image_count = 10
plot_images = correct_images[:image_count]
plot_labels = correct_pred_labels[:image_count]
plot_heatmaps = correct_heatmaps[:image_count]
plot_saliency = saliency_masks[:image_count]

plt.figure(figsize=(10, 40))

for i in range(image_count):
    plt.subplot(image_count, 3, 3*i + 1)
    plt.imshow(plot_images[i])
    plt.title(f"Image: {class_names[plot_labels[i]]}")
    plt.axis('off')

    plt.subplot(image_count, 3, 3*i + 2)
    plt.imshow(plot_heatmaps[i], cmap='jet')
    plt.title("Grad-Cam++ Heatmap")
    plt.axis('off')

    plt.subplot(image_count, 3, 3*i + 3)
    plt.imshow(plot_images[i])
    plt.imshow(plot_saliency[i], alpha=0.4, cmap='jet')
    plt.title("Saliency Mask")
    plt.axis('off')

plt.tight_layout()
plt.show()

```
import cv2
import numpy as np
import nibabel as nib

def save_jpg_as_nifti(jpg_path, nii_path):
    img = cv2.imread(jpg_path, cv2.IMREAD_GRAYSCALE) 
    img = img.astype(np.float32)

    affine = np.eye(4)

    nii_img = nib.Nifti1Image(img, affine)
    nib.save(nii_img, nii_path)
    print(f"Saved MRI NIfTI: {nii_path}")
```

```
def save_mask_nifti(mask, reference_nii_path, out_path):
    ref_nii = nib.load(reference_nii_path)
    affine = ref_nii.affine 

    mask_nii = nib.Nifti1Image(mask.astype(np.uint8), affine)
    nib.save(mask_nii, out_path)
    print(f"Saved mask: {out_path}")
```

```
import os
import cv2
import numpy as np
import nibabel as nib

output_dir = "radiomics_ready/"
os.makedirs(output_dir, exist_ok=True)

def save_jpg_as_nifti(jpg_path, nii_path):
    img = cv2.imread(jpg_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)

    affine = np.eye(4)            
    nii = nib.Nifti1Image(img, affine)

    nib.save(nii, nii_path)

def save_mask_nifti(mask, ref_nii_path, out_path):
    ref = nib.load(ref_nii_path)
    nii = nib.Nifti1Image(mask.astype(np.uint8), ref.affine)
    nib.save(nii, out_path)

for index, (img, heatmap, pred) in enumerate(zip(correct_images, correct_heatmaps, correct_pred_labels)):

    class_name = class_names[pred]
    jpg_path   = test_generator.filepaths[index]   
    print(f"Processing: {jpg_path}")

    mri_nii_path = os.path.join(output_dir, f"{index:03d}_{class_name}_MRI.nii.gz")
    save_jpg_as_nifti(jpg_path, mri_nii_path)

    if class_name != 'notumor':
        tumor_mask = generate_pseudo_mask(img)  
    else:
        tumor_mask = None

    saliency_mask = get_saliency_mask(heatmap, tumor_mask, percentile=90)

    mask_nii_path = os.path.join(output_dir, f"{index:03d}_{class_name}_saliency_mask.nii.gz")
    save_mask_nifti(saliency_mask, mri_nii_path, mask_nii_path)

    print(f"Saved: {mri_nii_path}  |  {mask_nii_path}")

```

In [ ]:
plt.style.use('default')
sns.set_theme(style="whitegrid")

df_comparsion = pd.read_csv("/kaggle/input/brain-tumor-classes/Class_radiomics_saliency_comparison.csv")

df_comparsion.head()

In [ ]:
df_clean = df_comparsion.copy() 

diag_cols = df_clean.filter(regex='diagnostics').columns
df_clean = df_clean.drop(columns=diag_cols)

df_clean = df_clean.select_dtypes(include=['float64', 'int64'])

print(df_clean.shape)
df_clean.head()

In [ ]:
df_clean["TumorType"] = df_comparsion["ID"].apply(lambda x: x.split("_")[1])
df_clean["TumorType"].value_counts()

In [ ]:
feature_cols = df_clean.filter(regex="IN_|OUT_").columns
X = df_clean[feature_cols]

selector = VarianceThreshold(threshold=0.01) 
X_reduced = selector.fit_transform(X)
selected_features = X.columns[selector.get_support()]

df_clean = df_clean[selected_features.tolist() + ["TumorType"]]
print(df_clean.shape)

In [ ]:
df_clean.info()
df_clean.head()

In [ ]:
results = []
in_features = [f for f in df_clean.columns if f.startswith("IN_")]

for tumor in df_clean['TumorType'].unique():
    df_tumor = df_clean[df_clean['TumorType'] == tumor]

    for in_feature in in_features:
        out_feature = in_feature.replace("IN_", "OUT_")
        if out_feature not in df_tumor.columns:
            continue

        x = df_tumor[in_feature].astype(float)
        y = df_tumor[out_feature].astype(float)

        mask = (~x.isna()) & (~y.isna()) & np.isfinite(x) & np.isfinite(y)
        x_valid, y_valid = x[mask], y[mask]

        if len(x_valid) < 3:  
            results.append({
                "TumorType": tumor,
                "Feature": in_feature,
                "N_pairs": len(x_valid),
                "Wilcoxon_stat": None,
                "P_value": None,
                "Median_IN": np.median(x_valid) if len(x_valid) > 0 else None,
                "Median_OUT": np.median(y_valid) if len(y_valid) > 0 else None,
                "Effect_Direction": "Higher_IN" if np.median(x_valid) > np.median(y_valid) else "Higher_OUT"
            })
            continue

        stat, pval = wilcoxon(x_valid, y_valid)

        results.append({
            "TumorType": tumor,
            "Feature": in_feature,
            "N_pairs": len(x_valid),
            "Wilcoxon_stat": stat,
            "P_value": pval,
            "Median_IN": np.median(x_valid),
            "Median_OUT": np.median(y_valid),
            "Effect_Direction": "Higher_IN" if np.median(x_valid) > np.median(y_valid) else "Higher_OUT"
        })

results_df = pd.DataFrame(results)
results_df.head()

In [ ]:
results_df["EffectSize"] = np.log2((results_df["Median_IN"] + 1e-8) / 
                                   (results_df["Median_OUT"] + 1e-8))  
results_df["NegLog10P"] = -np.log10(results_df["P_value"])

In [ ]:
plt.figure(figsize=(10,7))
sns.scatterplot(
    data=results_df,
    x="EffectSize",
    y="NegLog10P",
    hue="TumorType",
    size="NegLog10P",
    sizes=(20,200),
    alpha=0.8
)
plt.axhline(-np.log10(0.05), linestyle='--')
plt.title("Volcano Plot Across Tumor Types")
plt.xlabel("log2( Fold Change IN / OUT )")
plt.ylabel("-log10(P-value)")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

pivot = results_df.pivot_table(
    index="Feature", 
    columns="TumorType",
    values="Effect_Direction",
    aggfunc=lambda x: (x == "Higher_IN").mean() 
)

plt.figure(figsize=(14, 10))

sns.heatmap(
    pivot,
    annot=True,
    fmt=".2f",
    linewidths=0.5,
    linecolor='gray',
    cmap="viridis", 
    cbar_kws={"label": "Fraction where IN > OUT"}
)

plt.title("Fraction of Samples Where Feature IN > OUT (per Tumor Type)", fontsize=16)
plt.xlabel("Tumor Type", fontsize=14)
plt.ylabel("Radiomic Feature", fontsize=14)

plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=10) 

plt.tight_layout()
plt.show()

In [ ]:
sns.displot(data=results_df, x="EffectSize", hue="TumorType", kind="kde")
plt.axvline(0, linestyle="--")
plt.title("Effect Size Distribution — IN vs OUT saliency mask")
plt.show()

In [ ]:
sig = results_df[results_df["P_value"] < 0.01]  
strong = sig[abs(sig["EffectSize"]) > 1] 

# TOP TEXTURE BIOMARKERS PER TUMOR
top = strong.sort_values("P_value").groupby("TumorType").head(10)
top